In [21]:
import os
from numpy import linspace
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
import pathlib

In [22]:
reynolds_no = linspace(1, 2000, 80) # Reynolds numbers to simulate
# Kinematic viscosity values based on characteristic geometry
viscosity_vals = {
    "bend": 0.02 / reynolds_no,
    "step": 0.1 / reynolds_no,
    "channel": 0.1 / reynolds_no,
    "channelstrip": 0.1 / reynolds_no
}

### Create Case Directories

Get the name of the folders in Templates directory

In [23]:
path = pathlib.Path.cwd()
templates_dir = path.parent / 'templates'
templates_dir

PosixPath('/home/niko/Documents/Python/reynoldsDeepL/templates')

Iterate over the folders in templates dir to create flow cases

In [24]:
for dir in templates_dir.iterdir():
    templateDir = SolutionDirectory(dir, archive=None, paraviewLink=False)
    # Get the type of the case
    template_case_name = dir.stem
    # Drop the template
    case_name = template_case_name[:-8]
    for i, reynolds in enumerate(reynolds_no):
        # Define the unique name for the case
        flow_case = path.parent / "flowCases" / f"{case_name}Re={reynolds:.2f}"
        # Create a new case
        case = templateDir.cloneCase(str(flow_case), paraviewLink=False)
        if "bend" in case_name.lower():
            vis = viscosity_vals["bend"]
        elif "step" in case_name.lower():
            vis = viscosity_vals["step"]
        elif "channelstrip" in case_name.lower():
            vis = viscosity_vals["channelstrip"]
        else:
            vis = viscosity_vals["channel"]
        # Get the directory with viscosity parameter
        transportProps = ParsedParameterFile(flow_case / "constant" / "transportProperties")
        # Set appropriate value of viscosity
        transportProps["nu"] = vis[i]
        # Save the transport properties dictionary
        transportProps.writeFile()
             